# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [11]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [12]:
#Load the weather dataframe from WeatherPy and clean the dataframe to drop NaN values
weather_data_to_load = "../WeatherPy/output_data/cities.csv"
weather_data = pd.read_csv(weather_data_to_load)
del weather_data["Unnamed: 0"]
weather_data = weather_data.dropna()
weather_data


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,albany,42.60,-73.97,39.99,68,82,3.00,US,1609200607
1,faanui,-16.48,-151.75,80.92,69,10,7.61,PF,1609200836
2,odessa,46.48,30.73,46.40,100,90,15.66,UA,1609200836
3,college,64.86,-147.80,12.20,78,75,3.36,US,1609200837
4,busselton,-33.65,115.33,72.00,52,80,3.00,AU,1609200837
...,...,...,...,...,...,...,...,...,...
579,correntina,-13.34,-44.64,71.65,64,42,3.22,BR,1609200891
580,miri,4.41,114.01,77.00,94,40,5.82,MY,1609200891
581,bajil,15.06,43.29,73.18,74,0,1.66,YE,1609200891
582,orlik,52.52,99.83,-34.11,66,0,1.74,RU,1609200891


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [13]:
#Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [14]:
#Use the Lat and Lng as locations and Humidity as the weight
locations = weather_data[["Lat", "Lng"]].astype(float)
humidity = weather_data["Humidity"].astype(float)

In [19]:
#Create the map
figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

#Add the heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
